In [ ]:
import requests
import csv
import time
import base64
import json

In [ ]:
def extract_users(session, endpoint):
    response = session.get(endpoint)
    data = response.json()

    # Extract users' email and id and append them to a CSV file
    with open('zendesk_users.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for user in data['users']:
            email = user['email']
            user_id = user['id']
            writer.writerow([email, user_id])

    # Check if there is a next page
    if data['next_page']:
        time.sleep(0.2)
        next_page = data['next_page']
        print("Processing page:", next_page)
        extract_users(session, next_page)


In [ ]:
# Load configuration from config.json
with open('config.json') as config_file:
    config = json.load(config_file)

username = config['username'] + "/token"
password = config['api_token']
hostname = config['zndsk_subdomain']

In [ ]:
# Construct endpoint and headers for Basic Authentication
endpoint = f'https://{hostname}.zendesk.com/api/v2/users.json?page=1&role=end-user'

auth_string = f'{username}:{password}'
headers = {
    'Authorization': 'Basic ' + base64.b64encode(auth_string.encode()).decode()
}

In [ ]:
# Create a session
session = requests.Session()
session.headers.update(headers)

In [ ]:
# Start extraction from the first page
print("Processing page:", endpoint)
extract_users(session, endpoint)